In [2]:
## 模型与剖分

import math
import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from tool import PDE, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, H1Error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb

n   = 4  #剖分次数
n   = n + 1 
Lam = [1,1e1,1e2,1e3,1e4,1e5]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)
    
for i in range(len(Lam)):
    pde = PDE(1, Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NN = node.shape[0]
        NC = cell.shape[0]
        cm = np.ones(NC, dtype=np.float64) / NC
     
        # 单元刚度矩阵和单元载荷向量 
        A1, A2 = get_stiff_and_div_matrix(node, cell, cm)
        bb = get_bb(pde, node, cell, cm)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, node, cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, node, getIsBdNode)
        u = pde.solution(node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = H1Error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        node, cell = uniform_refine(node, cell)

if n-1 > 1: 
    for i in range(len(Lam)):
        fig = plt.figure()
        plt.plot(np.log(H[1:]), np.log(E[i][1:]))
        plt.title("lam={}".format(Lam[i]))
        plt.xlabel("log(h)")
        plt.ylabel("log(e)")
        plt.savefig(fname="Fem/elasticityFemLam_{}.png"
                    .format(Lam[i]))
        plt.close(fig)
        
    for i in range(len(Lam)):
        f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]) ,1)
        P[i] = f[0]

#print("u= ", np.max(u))
#print("uh= ", np.max(uh))
print_error(Lam, H, E)
#print_error(Lam, H, E[isInterNode], E_rel[isInterNode])
if n-1 > 1:
    print_P(Lam, P)

---------------------Lam= 1---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.11935462553808

h=  0.25
e= 0.06600434279613125

h=  0.125
e= 0.033524494459334836

h=  0.0625
e= 0.016823662478270256


---------------------Lam= 10.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.19490624029254422

h=  0.25
e= 0.09444701127327092

h=  0.125
e= 0.04677750841719942

h=  0.0625
e= 0.023372403831701003


---------------------Lam= 100.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.23774330387469403

h=  0.25
e= 0.11241654967463241

h=  0.125
e= 0.0542504566071042

h=  0.0625
e= 0.02656631822252743


---------------------Lam= 1000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.24412751547297354

h=  0.25
e= 0.11607902795574207

h=  0.125
e= 0.056243216427467754

h=  0.0625
e= 0.027059098623017558


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.24479742009342947

h=  0.25
e= 0.11649958091039243

h=  0.125
e= 0.056559074845972626

h=  0.0625
